In [1]:
import numpy as np
import cv2 as cv
from glob import glob


def angle_cos(p0, p1, p2):
    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

def reorder_vertices(vertices):
    x_min = np.min(vertices[:,0])
    x_max = np.max(vertices[:,0])
    y_min = np.min(vertices[:,1])
    y_max = np.max(vertices[:,1])
    new_vertices = np.zeros((4,2), dtype=np.int32)
    new_vertices[0,0], new_vertices[0,1] = x_min, y_min
    new_vertices[1,0], new_vertices[1,1] = x_max, y_min
    new_vertices[2,0], new_vertices[2,1] = x_max, y_max
    new_vertices[3,0], new_vertices[3,1] = x_min, y_max
    return new_vertices

def find_squares(img):
    img = cv.GaussianBlur(img, (5, 5), 0)
    squares = []
    for gray in cv.split(img):
        for thrs in range(0, 255, 26):
            if thrs == 0:
                bin = cv.Canny(gray, 0, 50, apertureSize=5)
                bin = cv.dilate(bin, None)
            else:
                _retval, bin = cv.threshold(gray, thrs, 255, cv.THRESH_BINARY)
            contours, _hierarchy = cv.findContours(bin, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                cnt_len = cv.arcLength(cnt, True)
                cnt = cv.approxPolyDP(cnt, 0.02*cnt_len, True)
                if len(cnt) == 4 and cv.contourArea(cnt) > 9000 and cv.contourArea(cnt) < 40000 and cv.isContourConvex(cnt):
#                     print('area: ', cv.contourArea(cnt))
                    cnt = cnt.reshape(-1, 2)
                    max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                    if max_cos < 0.1:
                        cnt = reorder_vertices(cnt)
                        squares.append(cnt)
    return squares

def remove_duplicates(squares):
    threshold = 50
    pos_y = [squares[i][0][1] for i in range(len(squares))]
    idx_y = np.argsort(pos_y)
    new_idx = []
    tmp_idx = []

    for i in range(len(idx_y)):
        if len(tmp_idx) == 0:
            tmp_idx.append(idx_y[i])
        if abs(pos_y[idx_y[i]] - pos_y[tmp_idx[-1]]) < threshold:
            if tmp_idx[-1] != idx_y[i]:
                tmp_idx.append(idx_y[i])
        else:
            pos_x = [squares[j][0][0] for j in tmp_idx]
            idx_x = np.argsort(pos_x)
            tmp2_idx = []

            for j in range(len(idx_x)):
                if len(tmp2_idx) == 0:
                    tmp2_idx.append(tmp_idx[idx_x[j]])
                if abs(squares[tmp_idx[idx_x[j]]][0][0] - squares[tmp2_idx[-1]][0][0] ) > threshold:
                    tmp2_idx.append(tmp_idx[idx_x[j]])
#             print(len(tmp2_idx))
            new_idx.extend(tmp2_idx)
            tmp_idx = []
    pos_x = [squares[j][0][0] for j in tmp_idx]
    idx_x = np.argsort(pos_x)
    tmp2_idx = []

    for j in range(len(idx_x)):
        if len(tmp2_idx) == 0:
            tmp2_idx.append(tmp_idx[idx_x[j]])
        if abs(squares[tmp_idx[idx_x[j]]][0][0] - squares[tmp2_idx[-1]][0][0] ) > threshold:
            tmp2_idx.append(tmp_idx[idx_x[j]])
#     print(len(tmp2_idx))
    new_idx.extend(tmp2_idx)
    new_squares = [squares[i] for i in new_idx]
    return new_squares

In [21]:
for fn in glob('./data/pic*.png'):
    img = cv.imread(fn)
    squares = find_squares(img)
    squares = remove_duplicates(squares)
    print('num squares: ', len(squares))
    cv.drawContours( img, squares, -1, (0, 255, 0), 3 )
    cv.imshow('squares', img)
    ch = cv.waitKey()
    if ch == 27:
        break
    break

print('Done')

cv.destroyAllWindows()

num squares:  30
Done


In [22]:
# for i in range(len(squares)):
#     print('area: ', cv.contourArea(squares[i]))

In [23]:
img.shape

(520, 960, 3)

In [24]:
# Crop to button
offset = 10
button_contour = [squares[28]]
button_contour[0][0,0], button_contour[0][0,1] = button_contour[0][0,0] + offset, button_contour[0][0,1] + offset
button_contour[0][1,0], button_contour[0][1,1] = button_contour[0][1,0] - offset, button_contour[0][1,1] + offset
button_contour[0][2,0], button_contour[0][2,1] = button_contour[0][2,0] - offset, button_contour[0][2,1] - offset
button_contour[0][3,0], button_contour[0][3,1] = button_contour[0][3,0] + offset, button_contour[0][3,1] - offset

img = cv.imread(fn)
mask = np.zeros_like(img) # Create mask where white is what we want, black otherwise
cv.drawContours(mask, button_contour, 0, 255, -1) # Draw filled contour in mask
out = np.zeros_like(img) # Extract out the object and place into output image

out[mask == 255] = img[mask == 255]

# Now crop
(y, x, _) = np.where(mask == 255)
(topy, topx) = (np.min(y), np.min(x))
(bottomy, bottomx) = (np.max(y), np.max(x))
out_im = img[topy:bottomy, topx:bottomx, :]

# Show the output image
cv.imshow('Output', out_im)
cv.waitKey(0)
cv.destroyAllWindows()

In [19]:
print(img.shape)
print(out_im.shape)

(520, 960, 3)
(82, 117, 3)


In [25]:
# Check if button is empty --> 96% pixels are white
num_white = np.sum((out_im[:,:,0] >= 200) & (out_im[:,:,1] >= 200) & (out_im[:,:,2] >= 200))
num_pixels = np.prod(out_im.shape[:2])
print(num_white)
print(num_pixels)
print(num_white / num_pixels)

7704
7704
1.0
